In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install ipympl
!pip install tf2crf


In [ ]:
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import save
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import datetime, os
import pandas as pd
from sklearn.utils import resample
from collections import Counter
import matplotlib.pyplot as plt
import plotly.express as px
import tensorflow as tf
from tf2crf import CRF
from tensorflow.keras.layers import Input, Embedding, Bidirectional, GRU, LSTM, Dense, Conv2D, Conv1D
from tensorflow.keras.models import Model
from tf2crf import CRF, ModelWithCRFLoss, ModelWithCRFLossDSCLoss 
from sklearn.metrics import classification_report
from numpy import transpose


In [ ]:
path = '/content/gdrive/My Drive/ECE 765/Project/TrainingData/'
import os
files = sorted(os.listdir(path))
i=0
x = pd.DataFrame()
x_time = pd.DataFrame()
y = pd.DataFrame()
y_time = pd.DataFrame()

print(sorted(files))
# files = files[0:9]
for f in files:
  temp = pd.read_csv(path+f,header=None)
  if 'x_time' in f:
    x_time = x_time.append(temp)
  elif 'y_time' in f:
    y_time = y_time.append(temp)
  elif 'x' in f:
    x = x.append(temp)
  else:
    result = pd.DataFrame(pd.np.repeat(temp.values,4,axis = 0))
    y = y.append(result,ignore_index=True)


In [ ]:
print(x_time.shape)
print(x.shape)
y = y[0:len(x)]
print(y.shape)

In [ ]:
x = x.rename(columns={0: "A", 1: "B", 2: "C", 3: "D", 4: "E", 5: "F"})
x['time'] = x_time
x['label'] = y
x_r=x.shape[0]
x_c=x.shape[1]

In [ ]:
''' Code block for sliding window '''
w_size=160
visible_data=np.array([])

for i in range(0, x_r, int(w_size/2)):
  if (i+w_size)<x_r:
    window_data=x.iloc[i:i+w_size,0:6].values.flatten()  
    window_label=x['label'][i:i+w_size].value_counts().idxmax()
    
    window_data=np.append(window_data, window_label)
    if visible_data.size==0:
      visible_data=np.hstack((visible_data, window_data))
    else:
      visible_data=np.vstack((visible_data, window_data))
print(visible_data.shape)

# # save to npy file
# path = '/content/gdrive/My Drive/ECE 765/Project/'
# save(path+'data_window.npy', visible_data)

In [ ]:
''' Code block for sliding window with 50% overlap for 160x6'''
w_size=160
visible_data=[]
#print(visible_data.shape)

for i in range(0, x_r, int(w_size/2)): #Slide throuhg all the rows with a 50% increment equivalent to 50% overlap 
  if (i+w_size)<x_r:
    window_data=x.iloc[i:i+w_size,0:6].values
    window_label=x['label'][i:i+w_size].value_counts().idxmax() #Chooses the mode value amongst the labels
    label=np.full((w_size,1), window_label)
    window_data=np.append(window_data,label,axis=1)
    visible_data.append(window_data)

visible_data=np.stack(visible_data, axis=0)
print(visible_data.shape)

# # save to npy file
path = '/content/gdrive/My Drive/ECE 765/Project/'
save(path+'data_window160.npy', visible_data)


In [ ]:
#@title
''' FOR NEW DATA '''
path = '/content/gdrive/My Drive/ECE 765/Project/TestData/'
import os
files = sorted(os.listdir(path))
i=0
x = pd.DataFrame()
x_time = pd.DataFrame()
y = pd.DataFrame()
y_time = pd.DataFrame()
print(len(files))
print(sorted(files))
# files = files[0:9]
files = files[0:3]

for f in files:
  temp = pd.read_csv(path+f,header=None)
  if 'x_time' in f:
    x_time = x_time.append(temp,ignore_index=True)
  elif 'y_time' in f:
    y_time = y_time.append(temp,ignore_index=True)
  elif 'x' in f:
    x = x.append(temp,ignore_index=True)
  else:
    result = pd.DataFrame(pd.np.repeat(temp.values,4,axis = 0))

    y = y.append(result,ignore_index=True)
# # save to npy file
# path = '/content/gdrive/My Drive/ECE 765/Project/'
# save(path+'subject_009.npy', visible_data)

In [ ]:
#@title
''' FOR NEW DATA '''

path = '/content/gdrive/My Drive/ECE 765/Project/TestData/'
import os
files = sorted(os.listdir(path))
i=0
x = pd.DataFrame()
x_time = pd.DataFrame()
y = pd.DataFrame()
y_time = pd.DataFrame()
print(len(files))
print(sorted(files))
# files = files[0:9]
files = files[3:6]

for f in files:
  temp = pd.read_csv(path+f,header=None)
  if 'x_time' in f:
    x_time = x_time.append(temp,ignore_index=True)
  elif 'y_time' in f:
    y_time = y_time.append(temp,ignore_index=True)
  elif 'x' in f:
    x = x.append(temp,ignore_index=True)
  else:
    result = pd.DataFrame(pd.np.repeat(temp.values,4,axis = 0))

    y = y.append(result,ignore_index=True)
# # save to npy file
# path = '/content/gdrive/My Drive/ECE 765/Project/'
# save(path+'subject_010.npy', visible_data)

In [ ]:
#@title
''' FOR NEW DATA '''

path = '/content/gdrive/My Drive/ECE 765/Project/TestData/'
import os
files = sorted(os.listdir(path))
i=0
x = pd.DataFrame()
x_time = pd.DataFrame()
y = pd.DataFrame()
y_time = pd.DataFrame()
print(len(files))
print(sorted(files))
# files = files[0:9]
files = files[6:9]

for f in files:
  temp = pd.read_csv(path+f,header=None)
  if 'x_time' in f:
    x_time = x_time.append(temp,ignore_index=True)
  elif 'y_time' in f:
    y_time = y_time.append(temp,ignore_index=True)
  elif 'x' in f:
    x = x.append(temp,ignore_index=True)
  else:
    result = pd.DataFrame(pd.np.repeat(temp.values,4,axis = 0))

    y = y.append(result,ignore_index=True)
# # save to npy file
# path = '/content/gdrive/My Drive/ECE 765/Project/'
# save(path+'subject_011.npy', visible_data)

In [ ]:
#@title
''' FOR NEW DATA '''

path = '/content/gdrive/My Drive/ECE 765/Project/TestData/'
import os
files = sorted(os.listdir(path))
i=0
x = pd.DataFrame()
x_time = pd.DataFrame()
y = pd.DataFrame()
y_time = pd.DataFrame()
print(len(files))
print(sorted(files))
# files = files[0:9]
files = files[9:12]

for f in files:
  temp = pd.read_csv(path+f,header=None)
  if 'x_time' in f:
    x_time = x_time.append(temp,ignore_index=True)
  elif 'y_time' in f:
    y_time = y_time.append(temp,ignore_index=True)
  elif 'x' in f:
    x = x.append(temp,ignore_index=True)
  else:
    result = pd.DataFrame(pd.np.repeat(temp.values,4,axis = 0))

    y = y.append(result,ignore_index=True)
# # save to npy file
# path = '/content/gdrive/My Drive/ECE 765/Project/'
# save(path+'subject_012.npy', visible_data)

In [ ]:
path = '/content/gdrive/My Drive/ECE 765/Project/'
data = np.load(path+'data_window.npy')
test_009 = np.load(path+'subject_009.npy')
test_010 = np.load(path+'subject_010.npy')
test_011 = np.load(path+'subject_011.npy')
test_012 = np.load(path+'subject_012.npy')
test_009_length = 9498
test_010_length = 12270
test_011_length = 12940
test_012_length = 11330

print(data.shape)
print(test_009.shape)
print(test_010.shape)
print(test_011.shape)
print(test_012.shape)

In [ ]:
''' Downsampling Code '''

df = pd.DataFrame(data = data)
# Counter({0: 12522, 3: 2590, 2: 933, 1: 726})

# Separate majority and minority classes
df_majority1 = df[df[960]==0]
df_majority2 = df[df[960]==2]
df_majority3 = df[df[960]==3]
df_minority = df[df[960]==1]

# Downsample majority class
df_majority_downsampled1 = resample(df_majority1, 
                                 replace=False,    # sample without replacement
                                 n_samples=2590,     # to match minority class
                                 random_state=123) # reproducible results

df_majority_downsampled2 = resample(df_majority2, 
                                 replace=False,    # sample without replacement
                                 n_samples=933,     # to match minority class
                                 random_state=123) # reproducible results

df_majority_downsampled3 = resample(df_majority3, 
                                 replace=False,    # sample without replacement
                                 n_samples=2590,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled1,df_majority_downsampled2,df_majority_downsampled3, df_minority])
 
data = df_downsampled.to_numpy()
print(data.shape)

In [ ]:
X = data[:,0:960]
y = data[:,960]
y = y.astype('int32')
print(set(y))

In [ ]:
scl = StandardScaler()
scl = scl.fit(X)
X = scl.transform(X)
test_009 = scl.transform(test_009)
test_010 = scl.transform(test_010)
test_011 = scl.transform(test_011)
test_012 = scl.transform(test_012)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
''' Without Down sampling '''

# class_weight = {0: 1.,
#                 1: 17.24,
#                 2: 13.42,
#                 3:4.83}

# With Down sampling


class_weight = {0: 1.,
                1: 3.56,
                2: 2.77,
                3:1.}

In [ ]:
original_dim = 960
intermediate_dim1 = 512
intermediate_dim2 = 256
latent_dim = 64

inputs = keras.Input(shape=(960,))
h = layers.Dense(intermediate_dim1, activation='relu')(inputs)
h1 = layers.Dense(intermediate_dim2, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h1)
z_log_sigma = layers.Dense(latent_dim)(h1)

from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])



In [ ]:
# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x2 = layers.Dense(intermediate_dim2, activation='relu')(latent_inputs)
x3 = layers.Dense(intermediate_dim1, activation='relu')(x2)
outputs = layers.Dense(original_dim, activation='sigmoid')(x3)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

In [ ]:
def scheduler(epoch, lr):
...   if epoch < 10:
...     return lr
...   else:
...     return lr * tf.math.exp(-0.1)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
reconstruction_loss = keras.losses.mean_squared_logarithmic_error(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
opt = keras.optimizers.Adam(learning_rate=0.001)
vae.compile(optimizer=opt)

In [ ]:
%load_ext tensorboard

In [ ]:
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
hist = vae.fit(x_train, x_train, epochs=10, batch_size=128,validation_data=(x_test, x_test),callbacks=[tensorboard_callback,callback])

In [ ]:
%tensorboard --logdir logs

In [ ]:
x_train_encoded = encoder.predict(x_train)
x_train_encoded = np.array(x_train_encoded)
x_train_encoded = x_train_encoded[0,:]
x_test_encoded = encoder.predict(x_test)
x_test_encoded = np.array(x_test_encoded)
x_test_encoded = x_test_encoded[0,:]

In [ ]:
num_classes = 4
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)
    intermediate_dim = 128
    gap = layers.Dense(intermediate_dim, kernel_initializer='uniform',activation='relu')(input_layer)
    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)
    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train_encoded.shape[1:])
# print(model.summary())
# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from tensorflow.keras import backend as K
epochs = 10
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5, min_lr=0.01
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, verbose=1),
]
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-3,
#     decay_steps=10000,
#     decay_rate=0.9)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train_encoded,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_test_encoded,y_test),
    verbose=1,
    class_weight = class_weight
)

In [ ]:
y_pred = model.predict(x_test_encoded, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)
    intermediate_dim1 = 512
    intermediate_dim2 = 256
    intermediate_dim3 = 64
    intermediate_dim4 = 32
    h = layers.Dense(intermediate_dim1, kernel_initializer='uniform',activation='relu')(input_layer)
    h1 = layers.Dense(intermediate_dim2,kernel_initializer='uniform', activation='relu')(h)
    h2 = layers.Dense(intermediate_dim3, kernel_initializer='uniform',activation='relu')(h1)
    gap = layers.Dense(intermediate_dim4, kernel_initializer='uniform',activation='relu')(h2)
    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)
    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train.shape[1:])
# print(model.summary())
# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 10
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, verbose=1),
]
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-3,
#     decay_steps=10000,
#     decay_rate=0.9)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_test,y_test),
    verbose=1,
    class_weight = class_weight
)

In [ ]:

y_pred = model.predict(x_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

In [ ]:
x_train_encoded.shape

In [ ]:
x_train_encoded_list = list(x_train_encoded)
y_train_list = list(y_train)
y_train_list = transpose([y_train_list]*64)
x_train_encoded_list = np.array(x_train_encoded_list)
y_train_list = np.array(y_train_list)
x_train_encoded_list = x_train_encoded_list.reshape((x_train_encoded_list.shape[0],x_train_encoded_list.shape[1],1))

x_test_encoded_list = list(x_test_encoded)
y_test_list = list(y_test)
y_test_list = transpose([y_test_list]*64)
x_test_encoded_list = np.array(x_test_encoded_list)
y_test_list = np.array(y_test_list)
x_test_encoded_list = x_test_encoded_list.reshape((x_test_encoded_list.shape[0],x_test_encoded_list.shape[1],1))


In [ ]:
inputs = Input(shape=(x_train_encoded_list.shape[1:]))
# output = Embedding(10, 1, trainable=True, mask_zero=True)(inputs)
# output = Embedding(10, 1, trainable=True, mask_zero=True)(inputs)

output =     Conv1D(64, 3, activation='relu',padding='same')(inputs)


# output = Bidirectional(LSTM(64, return_sequences=True))(output)
output = Bidirectional(LSTM(64, return_sequences=True))(output)

output = Dense(4, activation='softmax')(output)
crf = CRF(dtype='float32')
output = crf(output)
base_model = Model(inputs, output)
model = ModelWithCRFLoss(base_model)
model.compile(optimizer='adam')
# print(base_model.summary())


model.fit(x=x_train_encoded_list, y=y_train_list, epochs=2,validation_data=(x_test_encoded_list,y_test_list),class_weight = class_weight)
# keras.utils.plot_model(base_model)
keras.utils.plot_model(base_model, show_shapes=True)

In [ ]:
model.evaluate(x_test_encoded_list,y_test_list)
y_pred = model.predict(x_test_encoded, batch_size=64, verbose=1)


In [ ]:
#@title
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
#@title
subject_009_predict =  model.predict(test_009)
subject_010_predict =  model.predict(test_010)
subject_011_predict =  model.predict(test_011)
subject_012_predict =  model.predict(test_012)
labels_009 = np.argmax(subject_009_predict, axis=-1)   
labels_010 = np.argmax(subject_010_predict, axis=-1)   
labels_011 = np.argmax(subject_011_predict, axis=-1)   
labels_012 = np.argmax(subject_012_predict, axis=-1)   

In [ ]:
#@title

# Save subject_009_prediction
labels_009_ext = []
for i in range(labels_009.shape[0]):
  for j in range(40):
    labels_009_ext.append(labels_009[i])

labels_009_ext_final = []
for i in range(len(labels_009_ext)):
  if i%2 ==0:
    labels_009_ext_final.append(labels_009_ext[i])
diff = test_009_length - len(labels_009_ext_final)
for i in range(diff-1):
  labels_009_ext_final.append(0)
pd.DataFrame(labels_009_ext_final).to_csv('subject_009_01__y_prediction.csv', index=False)


In [ ]:
#@title

# Save subject_010_prediction

labels_010_ext = []
for i in range(labels_010.shape[0]):
  for j in range(40):
    labels_010_ext.append(labels_010[i])
print(labels_010_ext)
print(len(labels_010_ext))
print(Counter(labels_010_ext))
labels_010_ext_final = []
for i in range(len(labels_010_ext)):
  if i%2 ==0:
    labels_010_ext_final.append(labels_010_ext[i])
diff = test_010_length - len(labels_010_ext_final)
for i in range(diff-1):
  labels_010_ext_final.append(0)

pd.DataFrame(labels_010_ext_final).to_csv('subject_010_01__y_prediction.csv', index=False)


In [ ]:
#@title

# Save subject_011_prediction

labels_011_ext = []
for i in range(labels_011.shape[0]):
  for j in range(40):
    labels_011_ext.append(labels_011[i])
print(labels_011_ext)
print(len(labels_011_ext))
print(Counter(labels_011_ext))
labels_011_ext_final = []
for i in range(len(labels_011_ext)):
  if i%2 ==0:
    labels_011_ext_final.append(labels_011_ext[i])
diff = test_011_length - len(labels_011_ext_final)
for i in range(diff-1):
  labels_011_ext_final.append(0)
pd.DataFrame(labels_011_ext_final).to_csv('subject_011_01__y_prediction.csv', index=False)


In [ ]:
#@title

# Save subject_012_prediction

labels_012_ext = []
for i in range(labels_012.shape[0]):
  for j in range(40):
    labels_012_ext.append(labels_012[i])
print(labels_012_ext)
print(len(labels_012_ext))
print(Counter(labels_012_ext))
labels_012_ext_final = []
for i in range(len(labels_012_ext)):
  if i%2 ==0:
    labels_012_ext_final.append(labels_012_ext[i])
diff = test_012_length - len(labels_012_ext_final)
for i in range(diff-1):
  labels_012_ext_final.append(0)

pd.DataFrame(labels_012_ext_final).to_csv('subject_012_01__y_prediction.csv', index=False)
